In [2]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd


In [3]:

n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A_3.csv")
pi = pd.read_csv("pi_3.csv")

A = np.array(A)
pi = np.array(pi)
pi = pi.T
b = np.array([1000 for _ in range(m)])  


OffLine

In [4]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()
x.value


print(x.value)
print(prob.value)
opv1 = prob.value

x = x.value
x = np.where(x >= 0.5, 1, 0)

[3.50514653e-10 6.10855710e-10 1.37669013e-10 ... 1.49062147e-09
 1.09774524e-08 2.61829621e-10]
56275.0128878739


In [13]:
Anow = A
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)
piNow = np.reshape(piNow, (n,))
bNow = b
bNow = bNow.T

temp = m+n
y = cp.Variable(temp)
e = np.ones(n)
objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cp.Problem(objective, constraints)

prob.solve()
P = y[:m].value
p_truth = P
P

array([ 1.04581214,  2.05604972,  3.06239824,  4.07860401,  5.09664173,
        6.06381518,  7.06742707,  8.07094188,  9.0689332 , 10.08168534])

In [5]:
def one_time_learning(k):
    Anow = A[:, 0:k]
    Anow = Anow.T
    piNow = (pi.T)[0:k]
    piNow = np.reshape(piNow, (k,))
    bNow = b*(float(k/n))
    bNow = bNow.T

    temp = m+(k)
    y = cp.Variable(temp)
    e = np.ones(k)
    objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
    constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    P = y[:m].value
    AT = A.T
    xp50 = []
    xk50 = []
    for i in range(n):
        cri = (AT[i].dot(P))
        if(pi[0][i] > cri):
            xp50.append(1)
        else:
            xp50.append(0)  # xt(p) 10000
        if i < k:
            xk50.append(0)  #50
    for t in range(k,n):
        test = 1
        for j in range(m):
            temp1 = A[j,t]*xp50[t]
            temp2 = b[j] - (A[j, :t]).dot(np.array(xk50))
            if temp1 > temp2:
                test = 0
                break
        if test == 1:
            xk50.append(xp50[t])
        else:
            xk50.append(0)
    return xk50, P

In [5]:
k = 50
l_decision, dual_price = one_time_learning(k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k =", k)
print("competetive ratio is", ratio)
print("dual price is", dual_price)

when k = 50
competetive ratio is 0.9458699458199166
dual price is [1.35081704 2.0091937  2.8204548  4.17300415 5.10632907 6.00617544
 6.86463668 7.98026771 9.07001817 9.94393725]


In [6]:
k = 100
l_decision, dual_price = one_time_learning(k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k =", k)
print("competetive ratio is", ratio)
print("dual price is", dual_price)

when k = 100
competetive ratio is 0.8574431604281767
dual price is [1.23488944 1.89667522 2.78963516 4.23203398 4.99708955 6.00553983
 7.01494116 8.20822609 9.18253666 9.94101562]


In [7]:
k = 200
l_decision, dual_price = one_time_learning(k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k =", k)
print("competetive ratio is", ratio)
print("dual price is", dual_price)

when k = 200
competetive ratio is 0.9017767400763443
dual price is [1.21109318 2.04822188 2.81572949 4.2145155  5.09514834 6.11409644
 7.03873742 8.02835596 9.05932856 9.99874148]


In [12]:
dic_ratio = {}
dic_price = {}
l_norm = []
for k in range(50, 1000, 50):
    l_decision, dual_price = one_time_learning(k)
    opv2 = (pi.dot(l_decision))[0]
    ratio = opv2/opv1
    dic_ratio[k] = ratio
    dic_price[k] = dual_price
    print("when k = ", k)
    print("competetive ratio is", ratio)
    print("norm of difference of dual price", np.linalg.norm(dual_price-p_truth))
    l_norm.append(np.linalg.norm(dual_price-p_truth))

when k =  50
competetive ratio is 0.9458699458199166
norm of difference of dual price 0.48415817230224634
when k =  100
competetive ratio is 0.8574431604281767
norm of difference of dual price 0.4761228831912981
when k =  150
competetive ratio is 0.9074666100220018
norm of difference of dual price 0.3624434228013435
when k =  200
competetive ratio is 0.9017767400763443
norm of difference of dual price 0.3447252874077467
when k =  250
competetive ratio is 0.9080734057634501
norm of difference of dual price 0.30387346474198834
when k =  300
competetive ratio is 0.9505572228668956
norm of difference of dual price 0.1521165292166184
when k =  350
competetive ratio is 0.9390086465931194
norm of difference of dual price 0.17606697480274755
when k =  400
competetive ratio is 0.9520154833508238
norm of difference of dual price 0.13989035327012608
when k =  450
competetive ratio is 0.9270862668821129
norm of difference of dual price 0.14993559571622433
when k =  500
competetive ratio is 0.90974